In [3]:
import pandas as pd
from datetime import *
from pandas_datareader.data import DataReader
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import spacy
import os
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re

from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
from scipy.sparse import coo_matrix, hstack

In [4]:
## Test for 1 csv file first, later can loop thru all
df = pd.read_csv("../../Processed Data/Financial News/AAPL.csv")
company = "Apple"  ## MAKE SURE TO MAKE THIS DYNAMIC

df.head()

,Date,word_count,text_label,Label,Return,text_lem_lst,text_lem_str,sentiment_txtblob,positivity_sentiment_nltk,compound_sentiment_nltk,negativity_sentiment_nltk,neutral_sentiment_nltk
0,2010-01-05,449,"Apple Buys Quattro, an Ad Firm. While Google i...",1,0.001729,"['Apple', 'Buys', 'Quattro,', 'an', 'Ad', 'Fir...","Apple Buys Quattro, an Ad Firm. While Google ...",0.025000,0.000,0.0000,0.000,1.000
1,2010-01-06,440,"Apple Buys Quattro, an Ad Firm. Apple encroach...",-1,-0.015906,"['Apple', 'Buys', 'Quattro,', 'an', 'Ad', 'Fir...","Apple Buys Quattro, an Ad Firm. Apple encroac...",0.100000,0.000,-0.4019,0.072,0.928
2,2010-01-06,404,Putting the “i” in Apple. iSlate has plenty of...,-1,-0.015906,"['Putting', 'the', '“i”', 'in', 'Apple.', 'iSl...",Putting the “i” in Apple. iSlate have plenty ...,0.000000,0.000,0.0000,0.000,1.000
3,2010-01-19,294,Reading Apple’s Tea Leaves. Take one splatter-...,1,0.044238,"['Reading', 'Apple’s', 'Tea', 'Leaves.', 'Take...",Reading Apple’s Tea Leaves. Take one splatter...,0.500000,0.000,0.0000,0.000,1.000
4,2010-01-25,290,Laptop Sales Help Apple Top Forecasts. The con...,1,0.026903,"['Laptop', 'Sales', 'Help', 'Apple', 'Top', 'F...",Laptop Sales Help Apple Top Forecasts. The co...,0.466667,0.391,0.8720,0.000,0.609


In [5]:
df.drop_duplicates(subset="Date", inplace=True, keep="first")
df.reset_index(drop=True, inplace=True)

In [6]:



X_train, X_test, y_train, y_test = train_test_split(df[["text_label", 'word_count', 'sentiment_txtblob', 'positivity_sentiment_nltk',
                                                    'compound_sentiment_nltk', 'negativity_sentiment_nltk', 'neutral_sentiment_nltk']], 
                                                    df["Label"], random_state = 1)


cv = CountVectorizer(ngram_range=(1, 2), stop_words="english", analyzer="word", max_df=0.8)

X_train_vect = X_train["text_label"]
X_test_vect = X_test["text_label"]

X_train_dtm = cv.fit_transform(X_train_vect)
X_test_dtm = cv.transform(X_test_vect)

remaining_feats = np.array(X_train[['word_count', 'sentiment_txtblob', 'positivity_sentiment_nltk',
                'compound_sentiment_nltk', 'negativity_sentiment_nltk', 'neutral_sentiment_nltk']])
remaining_test_feats = np.array(X_test[['word_count', 'sentiment_txtblob', 'positivity_sentiment_nltk',
                'compound_sentiment_nltk', 'negativity_sentiment_nltk', 'neutral_sentiment_nltk']])




X_train_dtm = hstack(([X_train_dtm, remaining_feats]))
X_test_dtm = hstack(([X_test_dtm, remaining_test_feats]))
#X_train_dtm = np.hstack([X_train_dtm, np.array(X_train[['word_count', 'sentiment_txtblob']])])

In [7]:
# print the number of features that were generated
print('Features: ', X_train_dtm.shape[1])


nb = SVC(gamma='auto')
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)


# Get the training accuracy
print('Training Accuracy: ', metrics.accuracy_score(y_train, nb.predict(X_train_dtm)))
# print the accuracy of its predictions
print('Test Accuracy: ', metrics.accuracy_score(y_test, y_pred_class))

Features:  10806
Training Accuracy:  0.5714285714285714
Test Accuracy:  0.4567901234567901


In [8]:
# MNB = MultinomialNB()
# MNB.fit(X_train_dtm, y_train)

BNB = BernoulliNB()
BNB.fit(X_train_dtm, y_train)

LogReg = LogisticRegression()
LogReg.fit(X_train_dtm, y_train)

SGD = SGDClassifier()
SGD.fit(X_train_dtm, y_train)

SVC = SVC()
SVC.fit(X_train_dtm, y_train)

C:\Users\ksjag\Anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [9]:
svc_pred = SVC.predict(X_test_dtm)
#mnb_pred = MNB_clf.predict(X_test_dtm)
bnb_pred = BNB.predict(X_test_dtm)
logreg_pred = LogReg.predict(X_test_dtm)
sgd_pred = SGD.predict(X_test_dtm)

In [10]:
ensemble_pred = np.add(svc_pred, bnb_pred + logreg_pred + sgd_pred)/4
actual_rtns = df["Return"].loc[y_test.index]
model_rtns = actual_rtns * ensemble_pred
print("Returns: ", list(np.cumprod(1+model_rtns))[-1]*100, "%")

Returns:  95.60572385274119 %


In [11]:
this_pred = pd.DataFrame({"Apple": list(map(lambda x: 1 if x>= 0 else -1, ensemble_pred))})

In [12]:
this_pred

,Apple
0,-1
1,-1
2,-1
3,-1
4,1
5,1
6,1
7,1
8,1
9,1
